In [ ]:
#searching kg and bhp
#site: https://www.carfolio.com/search/
#search name
#click Sort descending
#choose max_found(2015:2019)
#choose name with the fewest character count from search term
#pick bhp and kg
#append info to python database







In [52]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException   

data = pd.read_excel('C:\\Users\\Patrice\\Desktop\\Classes\\PIR\\Dataset_V1.2.xlsx')
data.head()
carlist=[]

for name in data.DivCar:
    dict_name = {'name':name, 'kg':float(0),'bhp':float(0)}
    carlist.append(dict_name)



In [60]:

#definir le wedriver et la page web qui va nous intéresser

driver = webdriver.Chrome('C:\\Users\\Patrice\\Desktop\\chromedriver.exe') 
driver.get('https://www.carfolio.com/search/');


#pour chaque voiture dans le dictionnaire, on va faire une recherche

for car in carlist:
    in_y1       = driver.find_element_by_xpath("//div[3]/div/div/input")
    in_y2       = driver.find_element_by_xpath("//input[2]")
    in_name     = driver.find_element_by_xpath("//div/div/input")
    search_but  = driver.find_element_by_xpath("//p[29]/input")
    weight      = []
    bhp         = []
    
    
    #on cherche entre 2015 et 2019, et on cherche le nom de la voiture
    
    in_y1.send_keys('2015')
    in_y2.send_keys('2019')
    in_name.send_keys(car.get('name').lower())
    search_but.click()
    time.sleep(2)
    
    
    #cherche s'il y a au moins un resultat, sinon on passe à la prochaine recherche
    
    try:
        sort_year=driver.find_element_by_css_selector("[class^='longlist ui eight cards']")
    except NoSuchElementException:
        driver.get('https://www.carfolio.com/search/')
    
    
    #si il y a seulement un resultat on peut pas trier par an, ducoup on doit voir si le bouton existe, et puis l'executer
    
    try:
        sort_year=driver.find_element_by_xpath("//a/i").click()
    except NoSuchElementException:
        pass
    
    
    #chercher toute les informations differents qui nous interesse des résultats
    
    page = requests.get(driver.current_url)
    soup = bs(page.text, 'html.parser')
    
    for bhp_result in soup.findAll("abbr", {"title":"brake horsepower"}):
        bhp.append(bhp_result.next_sibling)
    
    for w_result in soup.findAll("span", {"class":"weight"}):
        weight.append(re.sub("\D","",str(w_result)))
    
    
    car.update({'bhp':np.mean(list(np.float_(bhp))), 'kg':np.mean(list(np.float_(weight)))})
    print(car)
    
    
    #repasse à une nouvelle recherche après avoir bien trouver des résultats
    driver.get('https://www.carfolio.com/search/')
    





#time.sleep(5)
#search_box = driver.find_element_by_name('q')
#search_box.send_keys('ChromeDriver')
#search_box.submit()



{'name': 'ACURA NSX', 'kg': 1725.0, 'bhp': 373.0}


WebDriverException: Message: chrome not reachable
  (Session info: chrome=80.0.3987.122)
